<a href="https://colab.research.google.com/github/MichaelGuaca/Proyecto-IA/blob/main/03_modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
# Importar librerías necesarias
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder


In [86]:
#IMPORTAR DATOS DE KAGGLE A COLAB
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

udea-ai4eng-20242.zip: Skipping, found more recently modified local copy (use --force to force download)


In [97]:
#DESCOMPRIMIR ARCHIVO
!unzip udea-ai4eng-20242.zip

Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [98]:
#CARGAR DATOS TRAIN CON PANDAS
train_data = pd.read_csv("train.csv")
train_data["RENDIMIENTO_GLOBAL"]

,RENDIMIENTO_GLOBAL
0,medio-alto
1,bajo
2,bajo
3,alto
4,medio-bajo
...,...
692495,medio-alto
692496,bajo
692497,medio-bajo
692498,bajo


In [99]:
#Explorar datos
print(train_data.info())
print(train_data['RENDIMIENTO_GLOBAL'].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   PERIODO                         692500 non-null  int64 
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object
 7   FAMI_TIENEINTERNET              665871 non-null  object
 8   FAMI_EDUCACIONPADRE             669322 non-null  object
 9   FAMI_EDUCACIONMADRE             668836 non-null  object
 10  ESTU_PAGOMATRICULAPROPIO        686002 non-null  object
 11  RENDIMIENTO_GLOBAL              692500 non-null  object
dtypes: int64(2), object(10)
memory

In [90]:
#Preprocesamiento de datos
#Manejar valores faltantes (rellenando con una categoría específica para columnas categóricas)
for col in train_data.columns:
    if train_data[col].dtype == 'object':
        train_data[col].fillna('Desconocido', inplace=True)
    else:
        train_data[col].fillna(train_data[col].median(), inplace=True)

<ipython-input-90-73dd2afa7a95>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data[col].fillna(train_data[col].median(), inplace=True)
<ipython-input-90-73dd2afa7a95>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tr

In [91]:
#Codificar variables categóricas usando LabelEncoder
categorical_cols = train_data.select_dtypes(include='object').columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    label_encoders[col] = le

In [92]:
#Separar características (X) y objetivo (y)
X = train_data.drop(['RENDIMIENTO_GLOBAL', 'ID', 'PERIODO'], axis=1)
y = train_data['RENDIMIENTO_GLOBAL']

In [93]:
#Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [94]:
#Entrenar el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [95]:
#Evaluar el modelo
y_pred = model.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))

Accuracy: 0.37734296028880865

Classification Report:
               precision    recall  f1-score   support

           0       0.50      0.53      0.51     35165
           1       0.40      0.43      0.42     34573
           2       0.29      0.26      0.28     34259
           3       0.30      0.28      0.29     34503

    accuracy                           0.38    138500
   macro avg       0.37      0.38      0.37    138500
weighted avg       0.37      0.38      0.37    138500



In [12]:
#Importancia de las características
importances = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_})
importances = importances.sort_values(by='Importance', ascending=False)
print("\nFeature Importances:\n", importances)


Feature Importances:
                           Feature  Importance
0             ESTU_PRGM_ACADEMICO    0.309469
6             FAMI_EDUCACIONPADRE    0.157193
1          ESTU_PRGM_DEPARTAMENTO    0.148211
7             FAMI_EDUCACIONMADRE    0.126452
3         ESTU_HORASSEMANATRABAJA    0.093144
2  ESTU_VALORMATRICULAUNIVERSIDAD    0.075287
4            FAMI_ESTRATOVIVIENDA    0.058039
8        ESTU_PAGOMATRICULAPROPIO    0.017737
5              FAMI_TIENEINTERNET    0.014469


In [138]:
#Guardar CSV
submission = pd.DataFrame({'ID': X_val.index, 'RENDIMIENTO_GLOBAL': y_pred})
submission['RENDIMIENTO_GLOBAL'] = label_encoders["RENDIMIENTO_GLOBAL"].inverse_transform(submission['RENDIMIENTO_GLOBAL'])
submission.to_csv("submission.csv", index=False)
print(submission)

            ID RENDIMIENTO_GLOBAL
0       137278               bajo
1       534550               alto
2       314034         medio-alto
3       418461               alto
4       521356               bajo
...        ...                ...
138495  457734               alto
138496  329976               alto
138497   55010               bajo
138498  194187         medio-bajo
138499  628411               bajo

[138500 rows x 2 columns]


In [128]:
!kaggle competitions submit -c udea-ai4eng-20242 -f /content/submission_1.csv -m "Primera Entrega"

100% 1.93M/1.93M [00:00<00:00, 10.1MB/s]
Successfully submitted to UDEA/ai4eng 20242 - Pruebas Saber Pro Colombia